In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
name_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_list_with_name.db')
annual_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_eps_annual.db')
quarter_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_eps_quarter.db')

In [3]:
name_df = pd.read_sql('TOTAL', con=name_engine)
name_list = name_df.Symbol.to_list()

In [12]:
selected_ticker = pd.DataFrame(columns=['symbol','q_chg_rate','a_chg_rate','update_time'])
selected_ticker

,symbol,q_chg_rate,a_chg_rate,updatetime


In [13]:
%%capture
pd.set_option('mode.chained_assignment', None)
for ticker in name_list:
    try:
        # quarter
        quarter_eps = pd.read_sql('{}'.format(ticker),con=quarter_engine)
        quarter_eps = quarter_eps.head(6)
        if str(quarter_eps.iloc[0].datetime)[:4] != '2022':
            continue
        elif quarter_eps.iloc[:3].eps.min() < 0:
            continue
        else:
            updatetime = quarter_eps.iloc[0].datetime
            length = len(quarter_eps)
            quarter_eps['q_chg_rate'] = 0.0
            for i in range(length-1):
                quarter_eps['q_chg_rate'].iloc[i]  = round(((quarter_eps.iloc[i].eps - quarter_eps.iloc[i+1].eps) / quarter_eps.iloc[i+1].eps),3).copy()
            quarter_chg_rate = round(quarter_eps.iloc[:3].q_chg_rate.mean(),4)
            # print('{} eps: {}'.format(ticker, quarter_chg_rate))

        # annual
        annual_eps = pd.read_sql('{}'.format(ticker),con=annual_engine)
        annual_eps = annual_eps.head(6)
        if str(annual_eps.iloc[0].datetime)[:4] != '2022':
            continue
        elif annual_eps.iloc[:3].eps.min() < 0:
            continue
        else:
            length = len(annual_eps)
            annual_eps['q_chg_rate'] = 0.0
            for i in range(length-1):
                annual_eps['q_chg_rate'].iloc[i]  = round(((annual_eps.iloc[i].eps - annual_eps.iloc[i+1].eps) / annual_eps.iloc[i+1].eps),3).copy()
            annual_chg_rate = round(annual_eps.iloc[:3].q_chg_rate.mean(),4)
            # print('{} eps: {}'.format(ticker, annual_chg_rate))
        
        if quarter_chg_rate > 0.25 and annual_chg_rate > 0.25:
            # print('{} by quarter is {}'.format(ticker, mean_chg_rate))
            temp = pd.DataFrame([[ticker, updatetime, quarter_chg_rate, annual_chg_rate]],columns=['symbol','update_time','q_chg_rate','a_chg_rate'])
            selected_ticker = pd.concat([selected_ticker, temp],ignore_index=True)
    except:
        print('{} is error'.format(ticker))

In [15]:
selected_ticker.drop(columns={'updatetime'},inplace=True)

In [16]:
selected_ticker

,symbol,q_chg_rate,a_chg_rate,update_time
0,AIR,0.3047,2.4937,2022-05-31
1,ASTL,0.3503,inf,2022-06-30
2,CALM,13.412,21.8147,2022-05-31
3,CTRN,0.5973,0.8277,2022-04-30
4,CVCO,0.3107,0.567,2022-06-30
5,EA,0.697,0.4447,2022-06-30
6,FIVE,1.134,0.376,2022-04-30
7,GIII,1.2087,2.2233,2022-04-30
8,JHX,0.455,0.2923,2022-06-30
9,LOW,0.3017,0.633,2022-07-31


In [ ]:
selected_ticker.to_csv('canslim_select.csv',index=None)

In [6]:
ticker_list = ['MSFT','AAPL','BABA']

In [8]:
for ticker in ticker_list:
    # quarter
    quarter_eps = pd.read_sql('{}'.format(ticker),con=quarter_engine)
    quarter_eps = quarter_eps.head(6)
    if str(quarter_eps.iloc[0].datetime)[:4] != '2022':
        continue
    elif quarter_eps.iloc[:3].eps.min() < 0:
        print('{} weak'.format(ticker))
        continue
    else:
        length = len(quarter_eps)
        quarter_eps['q_chg_rate'] = 0.0
        for i in range(length-1):
            quarter_eps['q_chg_rate'].iloc[i]  = round(((quarter_eps.iloc[i].eps - quarter_eps.iloc[i+1].eps) / quarter_eps.iloc[i+1].eps),3).copy()
        quarter_chg_rate = round(quarter_eps.iloc[:3].q_chg_rate.mean(),4)
        print('{} quarter eps: {}'.format(ticker, quarter_chg_rate))

    # annual
    annual_eps = pd.read_sql('{}'.format(ticker),con=annual_engine)
    annual_eps = annual_eps.head(6)
    if str(annual_eps.iloc[0].datetime)[:4] != '2022':
        continue
    elif annual_eps.iloc[:3].eps.min() < 0:
        continue
    else:
        length = len(annual_eps)
        annual_eps['q_chg_rate'] = 0.0
        for i in range(length-1):
            annual_eps['q_chg_rate'].iloc[i]  = round(((annual_eps.iloc[i].eps - annual_eps.iloc[i+1].eps) / annual_eps.iloc[i+1].eps),3).copy()
        annual_chg_rate = round(annual_eps.iloc[:3].q_chg_rate.mean(),4)
        print('{}  annual eps: {}'.format(ticker, annual_chg_rate))

    if quarter_chg_rate > 0.25 and annual_chg_rate > 0.25:
        # print('{} by quarter is {}'.format(ticker, mean_chg_rate))
        temp = pd.DataFrame([[ticker, quarter_chg_rate, annual_chg_rate]],columns=['symbol','q_chg_rate','a_chg_rate'])
        selected_ticker = pd.concat([selected_ticker, temp],ignore_index=True)

C:\Users\Rain\AppData\Local\Temp\ipykernel_4356\1899474877.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quarter_eps['q_chg_rate'].iloc[i]  = round(((quarter_eps.iloc[i].eps - quarter_eps.iloc[i+1].eps) / quarter_eps.iloc[i+1].eps),3).copy()
C:\Users\Rain\AppData\Local\Temp\ipykernel_4356\1899474877.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annual_eps['q_chg_rate'].iloc[i]  = round(((annual_eps.iloc[i].eps - annual_eps.iloc[i+1].eps) / annual_eps.iloc[i+1].eps),3).copy()


MSFT quarter eps: -0.0617
MSFT  annual eps: 0.245


C:\Users\Rain\AppData\Local\Temp\ipykernel_4356\1899474877.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quarter_eps['q_chg_rate'].iloc[i]  = round(((quarter_eps.iloc[i].eps - quarter_eps.iloc[i+1].eps) / quarter_eps.iloc[i+1].eps),3).copy()


AAPL quarter eps: 0.0733
BABA weak


In [9]:
selected_ticker

,symbol,q_chg_rate,a_chg_rate
